In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
df = pd.read_csv("../data/eurocup_2020.csv")
df.head()

,stage,date,pens,pens_home_score,pens_away_score,team_name_home,team_name_away,team_home_score,team_away_score,possession_home,possession_away,total_shots_home,total_shots_away,shots_on_target_home,shots_on_target_away,duels_won_home,duels_won_away,events_list,lineup_home,lineup_away
0,Final,11.07.2021,True,3,2,Italy,England,1,1,66%,34%,19,6,6,2,53%,47%,"[{'event_team': 'away', 'event_time': "" 2' "", ...","[{'Player_Name': 'Insigne', 'Player_Number': '...","[{'Player_Name': 'Kane', 'Player_Number': '9'}..."
1,Semi-finals,07.07.2021,False,False,False,England,Denmark,2,1,59%,41%,20,6,10,3,50%,50%,"[{'event_team': 'away', 'event_time': "" 30' "",...","[{'Player_Name': 'Kane', 'Player_Number': '9'}...","[{'Player_Name': 'Krogh Damsgaard', 'Player_Nu..."
2,Semi-finals,06.07.2021,True,4,2,Italy,Spain,1,1,29%,71%,7,16,4,5,49%,51%,"[{'event_team': 'away', 'event_time': "" 51' "",...","[{'Player_Name': 'Insigne', 'Player_Number': '...","[{'Player_Name': 'Torres', 'Player_Number': '1..."
3,Quarter-finals,03.07.2021,False,False,False,Ukraine,England,0,4,48%,52%,7,10,2,6,42%,59%,"[{'event_team': 'away', 'event_time': "" 4' "", ...","[{'Player_Name': 'Yaremchuk', 'Player_Number':...","[{'Player_Name': 'Kane', 'Player_Number': '9'}..."
4,Quarter-finals,03.07.2021,False,False,False,Czech Republic,Denmark,1,2,56%,44%,16,11,5,7,48%,52%,"[{'event_team': 'away', 'event_time': "" 5' "", ...","[{'Player_Name': 'Schick', 'Player_Number': '1...","[{'Player_Name': 'Dolberg', 'Player_Number': '..."


In [8]:
categoricas = df.select_dtypes(include=object).columns.values
categoricas

array(['stage', 'date', 'pens_home_score', 'pens_away_score',
       'team_name_home', 'team_name_away', 'possession_home',
       'possession_away', 'duels_won_home', 'duels_won_away',
       'events_list', 'lineup_home', 'lineup_away'], dtype=object)

In [15]:
participantes = list(df['team_name_home'].unique())
print(f" En este dataframe esta compuesto por :{participantes}")



 En este dataframe esta compuesto por :[' Italy ', ' England ', ' Ukraine ', ' Czech Republic ', ' Belgium ', ' Switzerland ', ' Sweden ', ' France ', ' Croatia ', ' Netherlands ', ' Wales ', ' Portugal ', ' Germany ', ' Slovakia ', ' Finland ', ' Russia ', ' North Macedonia ', ' Spain ', ' Hungary ', ' Denmark ', ' Turkey ', ' Poland ', ' Scotland ', ' Austria ']


In [20]:
def total_goles(equipo):
    goles_local = df[(df['team_name_home'] == f'{equipo}')]['team_home_score'].sum()
    goles_vist = df[(df['team_name_away'] == f'{equipo}')]['team_away_score'].sum()
    return int(goles_local + goles_vist)

goles_por_equipo = [total_goles(equipo) for equipo in participantes]
goles_por_equipo


[13, 11, 6, 6, 9, 8, 5, 7, 7, 8, 3, 7, 6, 2, 1, 2, 2, 13, 3, 12, 1, 4, 1, 5]

In [22]:
def total_goles(equipo):
    goles_local = df[(df['team_name_home'] == f'{equipo}')]['team_away_score'].sum()
    goles_vist = df[(df['team_name_away'] == f'{equipo}')]['team_home_score'].sum()
    return int(goles_local + goles_vist)

goles_encajados = [total_goles(equipo) for equipo in participantes]
goles_encajados 

[4, 2, 10, 4, 3, 9, 4, 6, 8, 4, 6, 7, 7, 7, 3, 7, 8, 6, 6, 7, 8, 6, 5, 5]

In [35]:
def tiros(equipo):
    tiros_local = df[(df['team_name_home'] == f'{equipo}')]['total_shots_home'].sum()
    tiros_vist = df[(df['team_name_away'] == f'{equipo}')]['total_shots_away'].sum()
    return int(tiros_local + tiros_vist)
tiros_total = [tiros(equipo) for equipo in participantes]
tiros_total

[127,
 63,
 51,
 56,
 48,
 67,
 41,
 56,
 42,
 52,
 39,
 51,
 49,
 24,
 19,
 25,
 33,
 115,
 19,
 92,
 40,
 40,
 41,
 53]

In [37]:
def penaltis(equipo):
    penaltis_local = int(df[(df['team_name_home'] == f'{equipo}') & (df['pens_home_score'] != 'False')]['pens_home_score'].sum())
    penaltis_vist = int(df[(df['team_name_away'] == f'{equipo}') & (df['pens_home_score'] != 'False')]['pens_away_score'].sum())
    return int(penaltis_local + penaltis_vist)
penaltis_total = [penaltis(equipo) for equipo in participantes]
penaltis_total

[34, 2, 0, 0, 0, 6, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 23, 0, 0, 0, 0, 0, 0]

In [38]:
equipos=(participantes)
total_goles=(goles_por_equipo)
encajados= (goles_encajados )
tiros = (tiros_total)
penaltis = (penaltis_total)
equipos, total_goles,total_encajados,tiros,penaltis = pd.DataFrame(equipos),pd.DataFrame(total_goles),pd.DataFrame(encajados),pd.DataFrame(tiros),pd.DataFrame(penaltis)

In [39]:
pd.concat([equipos,total_goles,total_encajados,tiros,penaltis], axis=1, ignore_index=True)
data_final = pd.concat([equipos,total_goles,total_encajados,tiros,penaltis], axis=1, ignore_index=True)
data_final

,0,1,2,3,4
0,Italy,13,4,127,34
1,England,11,2,63,2
2,Ukraine,6,10,51,0
3,Czech Republic,6,4,56,0
4,Belgium,9,3,48,0
5,Switzerland,8,9,67,6
6,Sweden,5,4,41,0
7,France,7,6,56,4
8,Croatia,7,8,42,0
9,Netherlands,8,4,52,0


In [40]:
data_final.columns

RangeIndex(start=0, stop=5, step=1)

In [41]:
data_final.columns = ['Paises', 'Goles Metidos', 'Goles Encajados', 'Tiros a Puerta', 'Penaltis Lanzados']
data_final.columns

Index(['Paises', 'Goles Metidos', 'Goles Encajados', 'Tiros a Puerta',
       'Penaltis Lanzados'],
      dtype='object')

In [42]:
data_final

,Paises,Goles Metidos,Goles Encajados,Tiros a Puerta,Penaltis Lanzados
0,Italy,13,4,127,34
1,England,11,2,63,2
2,Ukraine,6,10,51,0
3,Czech Republic,6,4,56,0
4,Belgium,9,3,48,0
5,Switzerland,8,9,67,6
6,Sweden,5,4,41,0
7,France,7,6,56,4
8,Croatia,7,8,42,0
9,Netherlands,8,4,52,0
